# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
chars[:100]

array([76, 13, 60,  0, 57, 24, 59, 61, 21, 53, 53, 53, 10, 60,  0,  0, 20,
       61, 17, 60, 77, 75, 32, 75, 24, 37, 61, 60, 59, 24, 61, 60, 32, 32,
       61, 60, 32, 75, 12, 24, 28, 61, 24, 81, 24, 59, 20, 61, 54, 43, 13,
       60,  0,  0, 20, 61, 17, 60, 77, 75, 32, 20, 61, 75, 37, 61, 54, 43,
       13, 60,  0,  0, 20, 61, 75, 43, 61, 75, 57, 37, 61, 30, 73, 43, 53,
       73, 60, 20, 42, 53, 53, 38, 81, 24, 59, 20, 57, 13, 75, 43], dtype=int32)

Now I need to split up the data into batches, and into training and validation sets. I should be making a test set here, but I'm not going to worry about that. My test will be if the network can generate new text.

Here I'll make both input and target arrays. The targets are the same as the inputs, except shifted one character over. I'll also drop the last bit of data so that I'll only have completely full batches.

The idea here is to make a 2D matrix where the number of rows is equal to the number of batches. Each row will be one long concatenated string from the character data. We'll split this data into a training set and validation set using the `split_frac` keyword. This will keep 90% of the batches in the training set, the other 10% in the validation set.

In [5]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the virst split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

In [6]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 200)

In [7]:
train_x.shape

(10, 178400)

In [8]:
train_x[:,:10]

array([[76, 13, 60,  0, 57, 24, 59, 61, 21, 53],
       [ 2, 43, 46, 61, 13, 24, 61, 77, 30, 81],
       [61, 64, 60, 57, 64, 13, 75, 43, 18, 61],
       [30, 57, 13, 24, 59, 61, 73, 30, 54, 32],
       [61, 57, 13, 24, 61, 32, 60, 43, 46, 63],
       [61, 44, 13, 59, 30, 54, 18, 13, 61, 32],
       [57, 61, 57, 30, 53, 46, 30, 42, 53, 53],
       [30, 61, 13, 24, 59, 37, 24, 32, 17, 31],
       [13, 60, 57, 61, 75, 37, 61, 57, 13, 24],
       [24, 59, 37, 24, 32, 17, 61, 60, 43, 46]], dtype=int32)

I'll write another function to grab batches out of the arrays made by split data. Here each batch will be a sliding window on these arrays with size `batch_size X num_steps`. For example, if we want our network to train on a sequence of 100 characters, `num_steps = 100`. For the next batch, we'll shift this window the next sequence of `num_steps` characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [9]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

In [10]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
        
    if sampling == True:
        batch_size, num_steps = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
        x_one_hot = tf.one_hot(inputs, num_classes, name='x_one_hot')
    
    with tf.name_scope('targets'):
        targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
        y_one_hot = tf.one_hot(targets, num_classes, name='y_one_hot')
        y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # Build the RNN layers
    with tf.name_scope("RNN_cells"):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    
    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)

    # Run the data through the RNN layers
    with tf.name_scope("RNN_forward"):
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)
    
    final_state = state
    
    # Reshape output so it's a bunch of rows, one row for each cell output
    with tf.name_scope('sequence_reshape'):
        seq_output = tf.concat(outputs, axis=1,name='seq_output')
        output = tf.reshape(seq_output, [-1, lstm_size], name='graph_output')
    
    # Now connect the RNN outputs to a softmax layer and calculate the cost
    with tf.name_scope('logits'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1),
                               name='softmax_w')
        softmax_b = tf.Variable(tf.zeros(num_classes), name='softmax_b')
        logits = tf.matmul(output, softmax_w) + softmax_b
        tf.summary.histogram('softmax_w', softmax_w)
        tf.summary.histogram('softmax_b', softmax_b)

    with tf.name_scope('predictions'):
        preds = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram('predictions', preds)
    
    with tf.name_scope('cost'):
        loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped, name='loss')
        cost = tf.reduce_mean(loss, name='cost')
        tf.summary.scalar('cost', cost)

    # Optimizer for training, using gradient clipping to control exploding gradients
    with tf.name_scope('train'):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    merged = tf.summary.merge_all()
    
    # Export the nodes 
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer', 'merged']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Hyperparameters

Here I'm defining the hyperparameters for the network. The two you probably haven't seen before are `lstm_size` and `num_layers`. These set the number of hidden units in the LSTM layers and the number of LSTM layers, respectively. Of course, making these bigger will improve the network's performance but you'll have to watch out for overfitting. If your validation loss is much larger than the training loss, you're probably overfitting. Decrease the size of the network or decrease the dropout keep probability.

In [11]:
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001

## Training

Time for training which is is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

In [12]:
!mkdir -p checkpoints/anna

In [13]:
def train(model, epochs, file_writer):
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # Use the line below to load a checkpoint and resume training
        #saver.restore(sess, 'checkpoints/anna20.ckpt')

        n_batches = int(train_x.shape[1]/num_steps)
        iterations = n_batches * epochs
        for e in range(epochs):

            # Train network
            new_state = sess.run(model.initial_state)
            loss = 0
            for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
                iteration = e*n_batches + b
                start = time.time()
                feed = {model.inputs: x,
                        model.targets: y,
                        model.keep_prob: 0.5,
                        model.initial_state: new_state}
                summary, batch_loss, new_state, _ = sess.run([model.merged, model.cost, 
                                                              model.final_state, model.optimizer], 
                                                              feed_dict=feed)
                loss += batch_loss
                end = time.time()
                print('Epoch {}/{} '.format(e+1, epochs),
                      'Iteration {}/{}'.format(iteration, iterations),
                      'Training loss: {:.4f}'.format(loss/b),
                      '{:.4f} sec/batch'.format((end-start)))

                file_writer.add_summary(summary, iteration)

In [14]:
epochs = 20
batch_size = 100
num_steps = 100
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

for lstm_size in [128,256,512]:
    for num_layers in [1, 2]:
        for learning_rate in [0.002, 0.001]:
            log_string = 'logs/4/lr={},rl={},ru={}'.format(learning_rate, num_layers, lstm_size)
            writer = tf.summary.FileWriter(log_string)
            model = build_rnn(len(vocab), 
                    batch_size=batch_size,
                    num_steps=num_steps,
                    learning_rate=learning_rate,
                    lstm_size=lstm_size,
                    num_layers=num_layers)
            
            train(model, epochs, writer)

Epoch 1/20  Iteration 1/3560 Training loss: 4.4216 0.6137 sec/batch
Epoch 1/20  Iteration 2/3560 Training loss: 4.4110 0.4663 sec/batch
Epoch 1/20  Iteration 3/3560 Training loss: 4.3990 0.4647 sec/batch
Epoch 1/20  Iteration 4/3560 Training loss: 4.3810 0.4547 sec/batch
Epoch 1/20  Iteration 5/3560 Training loss: 4.3435 0.4454 sec/batch
Epoch 1/20  Iteration 6/3560 Training loss: 4.2762 0.4482 sec/batch
Epoch 1/20  Iteration 7/3560 Training loss: 4.1987 0.4440 sec/batch
Epoch 1/20  Iteration 8/3560 Training loss: 4.1223 0.4513 sec/batch
Epoch 1/20  Iteration 9/3560 Training loss: 4.0532 0.4550 sec/batch
Epoch 1/20  Iteration 10/3560 Training loss: 3.9925 0.4518 sec/batch
Epoch 1/20  Iteration 11/3560 Training loss: 3.9369 0.4550 sec/batch
Epoch 1/20  Iteration 12/3560 Training loss: 3.8891 0.4496 sec/batch
Epoch 1/20  Iteration 13/3560 Training loss: 3.8473 0.4525 sec/batch
Epoch 1/20  Iteration 14/3560 Training loss: 3.8110 0.4490 sec/batch
Epoch 1/20  Iteration 15/3560 Training loss

Epoch 1/20  Iteration 120/3560 Training loss: 3.1058 0.4659 sec/batch
Epoch 1/20  Iteration 121/3560 Training loss: 3.1029 0.4656 sec/batch
Epoch 1/20  Iteration 122/3560 Training loss: 3.0998 0.5070 sec/batch
Epoch 1/20  Iteration 123/3560 Training loss: 3.0967 0.4707 sec/batch
Epoch 1/20  Iteration 124/3560 Training loss: 3.0937 0.4747 sec/batch
Epoch 1/20  Iteration 125/3560 Training loss: 3.0907 0.4682 sec/batch
Epoch 1/20  Iteration 126/3560 Training loss: 3.0875 0.4644 sec/batch
Epoch 1/20  Iteration 127/3560 Training loss: 3.0845 0.4664 sec/batch
Epoch 1/20  Iteration 128/3560 Training loss: 3.0817 0.4798 sec/batch
Epoch 1/20  Iteration 129/3560 Training loss: 3.0786 0.4791 sec/batch
Epoch 1/20  Iteration 130/3560 Training loss: 3.0757 0.4660 sec/batch
Epoch 1/20  Iteration 131/3560 Training loss: 3.0728 0.4622 sec/batch
Epoch 1/20  Iteration 132/3560 Training loss: 3.0698 0.4659 sec/batch
Epoch 1/20  Iteration 133/3560 Training loss: 3.0668 0.4660 sec/batch
Epoch 1/20  Iteratio

Epoch 2/20  Iteration 238/3560 Training loss: 2.4786 0.4602 sec/batch
Epoch 2/20  Iteration 239/3560 Training loss: 2.4776 0.4665 sec/batch
Epoch 2/20  Iteration 240/3560 Training loss: 2.4768 0.4611 sec/batch
Epoch 2/20  Iteration 241/3560 Training loss: 2.4763 0.4668 sec/batch
Epoch 2/20  Iteration 242/3560 Training loss: 2.4754 0.4612 sec/batch
Epoch 2/20  Iteration 243/3560 Training loss: 2.4743 0.4661 sec/batch
Epoch 2/20  Iteration 244/3560 Training loss: 2.4739 0.4653 sec/batch
Epoch 2/20  Iteration 245/3560 Training loss: 2.4733 0.4676 sec/batch
Epoch 2/20  Iteration 246/3560 Training loss: 2.4721 0.4681 sec/batch
Epoch 2/20  Iteration 247/3560 Training loss: 2.4712 0.4711 sec/batch
Epoch 2/20  Iteration 248/3560 Training loss: 2.4706 0.4678 sec/batch
Epoch 2/20  Iteration 249/3560 Training loss: 2.4698 0.4679 sec/batch
Epoch 2/20  Iteration 250/3560 Training loss: 2.4693 0.4636 sec/batch
Epoch 2/20  Iteration 251/3560 Training loss: 2.4686 0.4661 sec/batch
Epoch 2/20  Iteratio

Epoch 2/20  Iteration 356/3560 Training loss: 2.4107 0.4676 sec/batch
Epoch 3/20  Iteration 357/3560 Training loss: 2.3704 0.4751 sec/batch
Epoch 3/20  Iteration 358/3560 Training loss: 2.3339 0.4627 sec/batch
Epoch 3/20  Iteration 359/3560 Training loss: 2.3261 0.4657 sec/batch
Epoch 3/20  Iteration 360/3560 Training loss: 2.3234 0.4723 sec/batch
Epoch 3/20  Iteration 361/3560 Training loss: 2.3227 0.4680 sec/batch
Epoch 3/20  Iteration 362/3560 Training loss: 2.3222 0.4673 sec/batch
Epoch 3/20  Iteration 363/3560 Training loss: 2.3228 0.4750 sec/batch
Epoch 3/20  Iteration 364/3560 Training loss: 2.3241 0.4619 sec/batch
Epoch 3/20  Iteration 365/3560 Training loss: 2.3250 0.4693 sec/batch
Epoch 3/20  Iteration 366/3560 Training loss: 2.3257 0.4655 sec/batch
Epoch 3/20  Iteration 367/3560 Training loss: 2.3242 0.4733 sec/batch
Epoch 3/20  Iteration 368/3560 Training loss: 2.3238 0.4838 sec/batch
Epoch 3/20  Iteration 369/3560 Training loss: 2.3238 0.4684 sec/batch
Epoch 3/20  Iteratio

Epoch 3/20  Iteration 474/3560 Training loss: 2.2861 0.4627 sec/batch
Epoch 3/20  Iteration 475/3560 Training loss: 2.2860 0.4651 sec/batch
Epoch 3/20  Iteration 476/3560 Training loss: 2.2857 0.4687 sec/batch
Epoch 3/20  Iteration 477/3560 Training loss: 2.2855 0.4719 sec/batch
Epoch 3/20  Iteration 478/3560 Training loss: 2.2851 0.4657 sec/batch
Epoch 3/20  Iteration 479/3560 Training loss: 2.2847 0.4690 sec/batch
Epoch 3/20  Iteration 480/3560 Training loss: 2.2846 0.4663 sec/batch
Epoch 3/20  Iteration 481/3560 Training loss: 2.2842 0.4645 sec/batch
Epoch 3/20  Iteration 482/3560 Training loss: 2.2838 0.4674 sec/batch
Epoch 3/20  Iteration 483/3560 Training loss: 2.2836 0.4646 sec/batch
Epoch 3/20  Iteration 484/3560 Training loss: 2.2834 0.4781 sec/batch
Epoch 3/20  Iteration 485/3560 Training loss: 2.2831 0.4886 sec/batch
Epoch 3/20  Iteration 486/3560 Training loss: 2.2829 0.4837 sec/batch
Epoch 3/20  Iteration 487/3560 Training loss: 2.2826 0.4744 sec/batch
Epoch 3/20  Iteratio

...

Epoch 18/20  Iteration 3143/3560 Training loss: 1.8482 0.4640 sec/batch
Epoch 18/20  Iteration 3144/3560 Training loss: 1.8482 0.4999 sec/batch
Epoch 18/20  Iteration 3145/3560 Training loss: 1.8482 0.5363 sec/batch
Epoch 18/20  Iteration 3146/3560 Training loss: 1.8481 0.5142 sec/batch
Epoch 18/20  Iteration 3147/3560 Training loss: 1.8481 0.4908 sec/batch
Epoch 18/20  Iteration 3148/3560 Training loss: 1.8478 0.4892 sec/batch
Epoch 18/20  Iteration 3149/3560 Training loss: 1.8475 0.5139 sec/batch
Epoch 18/20  Iteration 3150/3560 Training loss: 1.8477 0.4794 sec/batch
Epoch 18/20  Iteration 3151/3560 Training loss: 1.8477 0.4738 sec/batch
Epoch 18/20  Iteration 3152/3560 Training loss: 1.8474 0.4721 sec/batch
Epoch 18/20  Iteration 3153/3560 Training loss: 1.8475 0.4673 sec/batch
Epoch 18/20  Iteration 3154/3560 Training loss: 1.8475 0.4755 sec/batch
Epoch 18/20  Iteration 3155/3560 Training loss: 1.8474 0.4738 sec/batch
Epoch 18/20  Iteration 3156/3560 Training loss: 1.8473 0.4955 se

Epoch 19/20  Iteration 3257/3560 Training loss: 1.8359 0.5052 sec/batch
Epoch 19/20  Iteration 3258/3560 Training loss: 1.8359 0.5303 sec/batch
Epoch 19/20  Iteration 3259/3560 Training loss: 1.8357 0.4834 sec/batch
Epoch 19/20  Iteration 3260/3560 Training loss: 1.8359 0.4683 sec/batch
Epoch 19/20  Iteration 3261/3560 Training loss: 1.8363 0.4763 sec/batch
Epoch 19/20  Iteration 3262/3560 Training loss: 1.8363 0.4747 sec/batch
Epoch 19/20  Iteration 3263/3560 Training loss: 1.8359 0.4755 sec/batch
Epoch 19/20  Iteration 3264/3560 Training loss: 1.8363 0.4633 sec/batch
Epoch 19/20  Iteration 3265/3560 Training loss: 1.8362 0.4768 sec/batch
Epoch 19/20  Iteration 3266/3560 Training loss: 1.8372 0.4761 sec/batch
Epoch 19/20  Iteration 3267/3560 Training loss: 1.8375 0.4969 sec/batch
Epoch 19/20  Iteration 3268/3560 Training loss: 1.8377 0.4678 sec/batch
Epoch 19/20  Iteration 3269/3560 Training loss: 1.8374 0.5042 sec/batch
Epoch 19/20  Iteration 3270/3560 Training loss: 1.8377 0.5323 se

Epoch 19/20  Iteration 3371/3560 Training loss: 1.8310 0.4941 sec/batch
Epoch 19/20  Iteration 3372/3560 Training loss: 1.8313 0.4648 sec/batch
Epoch 19/20  Iteration 3373/3560 Training loss: 1.8313 0.4845 sec/batch
Epoch 19/20  Iteration 3374/3560 Training loss: 1.8312 0.4673 sec/batch
Epoch 19/20  Iteration 3375/3560 Training loss: 1.8311 0.4873 sec/batch
Epoch 19/20  Iteration 3376/3560 Training loss: 1.8310 0.4866 sec/batch
Epoch 19/20  Iteration 3377/3560 Training loss: 1.8310 0.4693 sec/batch
Epoch 19/20  Iteration 3378/3560 Training loss: 1.8310 0.4538 sec/batch
Epoch 19/20  Iteration 3379/3560 Training loss: 1.8311 0.4533 sec/batch
Epoch 19/20  Iteration 3380/3560 Training loss: 1.8310 0.4549 sec/batch
Epoch 19/20  Iteration 3381/3560 Training loss: 1.8309 0.4574 sec/batch
Epoch 19/20  Iteration 3382/3560 Training loss: 1.8309 0.4546 sec/batch
Epoch 20/20  Iteration 3383/3560 Training loss: 1.8979 0.4565 sec/batch
Epoch 20/20  Iteration 3384/3560 Training loss: 1.8613 0.4701 se

Epoch 20/20  Iteration 3485/3560 Training loss: 1.8172 0.4695 sec/batch
Epoch 20/20  Iteration 3486/3560 Training loss: 1.8170 0.4630 sec/batch
Epoch 20/20  Iteration 3487/3560 Training loss: 1.8168 0.4727 sec/batch
Epoch 20/20  Iteration 3488/3560 Training loss: 1.8167 0.4618 sec/batch
Epoch 20/20  Iteration 3489/3560 Training loss: 1.8167 0.4714 sec/batch
Epoch 20/20  Iteration 3490/3560 Training loss: 1.8168 0.4715 sec/batch
Epoch 20/20  Iteration 3491/3560 Training loss: 1.8168 0.4835 sec/batch
Epoch 20/20  Iteration 3492/3560 Training loss: 1.8167 0.5330 sec/batch
Epoch 20/20  Iteration 3493/3560 Training loss: 1.8166 0.5402 sec/batch
Epoch 20/20  Iteration 3494/3560 Training loss: 1.8165 0.5097 sec/batch
Epoch 20/20  Iteration 3495/3560 Training loss: 1.8165 0.4730 sec/batch
Epoch 20/20  Iteration 3496/3560 Training loss: 1.8163 0.5279 sec/batch
Epoch 20/20  Iteration 3497/3560 Training loss: 1.8161 0.5106 sec/batch
Epoch 20/20  Iteration 3498/3560 Training loss: 1.8157 0.4587 se

ValueError: Dimensions must be equal, but are 256 and 211 for 'RNN_forward/rnn/while/rnn/multi_rnn_cell/cell_0/cell_0/basic_lstm_cell/MatMul_1' (op: 'MatMul') with input shapes: [100,256], [211,512].

In [35]:
tf.train.get_checkpoint_state('checkpoints/anna')

model_checkpoint_path: "checkpoints/anna/i3560_l512_1.122.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i200_l512_2.432.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i400_l512_1.980.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i600_l512_1.750.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i800_l512_1.595.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1000_l512_1.484.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1200_l512_1.407.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1400_l512_1.349.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1600_l512_1.292.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i1800_l512_1.255.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i2000_l512_1.224.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i2200_l512_1.204.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i2400_l512_1.187.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i2600_l512_1.172.ckpt"
all_model_checkpoint_paths: "checkpoints/an

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [17]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [41]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    prime = "Far"
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [44]:
checkpoint = "checkpoints/anna/i3560_l512_1.122.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

Farlathit that if had so
like it that it were. He could not trouble to his wife, and there was
anything in them of the side of his weaky in the creature at his forteren
to him.

"What is it? I can't bread to those," said Stepan Arkadyevitch. "It's not
my children, and there is an almost this arm, true it mays already,
and tell you what I have say to you, and was not looking at the peasant,
why is, I don't know him out, and she doesn't speak to me immediately, as
you would say the countess and the more frest an angelembre, and time and
things's silent, but I was not in my stand that is in my head. But if he
say, and was so feeling with his soul. A child--in his soul of his
soul of his soul. He should not see that any of that sense of. Here he
had not been so composed and to speak for as in a whole picture, but
all the setting and her excellent and society, who had been delighted
and see to anywing had been being troed to thousand words on them,
we liked him.

That set in her money at th

In [43]:
checkpoint = "checkpoints/anna/i200_l512_2.432.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farnt him oste wha sorind thans tout thint asd an sesand an hires on thime sind thit aled, ban thand and out hore as the ter hos ton ho te that, was tis tart al the hand sostint him sore an tit an son thes, win he se ther san ther hher tas tarereng,.

Anl at an ades in ond hesiln, ad hhe torers teans, wast tar arering tho this sos alten sorer has hhas an siton ther him he had sin he ard ate te anling the sosin her ans and
arins asd and ther ale te tot an tand tanginge wath and ho ald, so sot th asend sat hare sother horesinnd, he hesense wing ante her so tith tir sherinn, anded and to the toul anderin he sorit he torsith she se atere an ting ot hand and thit hhe so the te wile har
ens ont in the sersise, and we he seres tar aterer, to ato tat or has he he wan ton here won and sen heren he sosering, to to theer oo adent har herere the wosh oute, was serild ward tous hed astend..

I's sint on alt in har tor tit her asd hade shithans ored he talereng an soredendere tim tot hees. Tise sor 

In [46]:
checkpoint = "checkpoints/anna/i600_l512_1.750.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Fard as astice her said he celatice of to seress in the raice, and to be the some and sere allats to that said to that the sark and a cast a the wither ald the pacinesse of her had astition, he said to the sount as she west at hissele. Af the cond it he was a fact onthis astisarianing.


"Or a ton to to be that's a more at aspestale as the sont of anstiring as
thours and trey.

The same wo dangring the
raterst, who sore and somethy had ast out an of his book. "We had's beane were that, and a morted a thay he had to tere. Then to
her homent andertersed his his ancouted to the pirsted, the soution for of the pirsice inthirgest and stenciol, with the hard and and
a colrice of to be oneres,
the song to this anderssad.
The could ounterss the said to serom of
soment a carsed of sheres of she
torded
har and want in their of hould, but
her told in that in he tad a the same to her. Serghing an her has and with the seed, and the camt ont his about of the
sail, the her then all houg ant or to hus

In [47]:
checkpoint = "checkpoints/anna/i1000_l512_1.484.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farrat, his felt has at it.

"When the pose ther hor exceed
to his sheant was," weat a sime of his sounsed. The coment and the facily that which had began terede a marilicaly whice whether the pose of his hand, at she was alligated herself the same on she had to
taiking to his forthing and streath how to hand
began in a lang at some at it, this he cholded not set all her. "Wo love that is setthing. Him anstering as seen that."

"Yes in the man that say the mare a crances is it?" said Sergazy Ivancatching. "You doon think were somether is ifficult of a mone of
though the most at the countes that the
mean on the come to say the most, to
his feesing of
a man she, whilo he
sained and well, that he would still at to said. He wind at his for the sore in the most
of hoss and almoved to see him. They have betine the sumper into at he his stire, and what he was that at the so steate of the
sound, and shin should have a geest of shall feet on the conderation to she had been at that imporsing the